In [32]:
# This example was taken from the langchain docs:
# https://python.langchain.com/v0.2/docs/tutorials/rag/
# %pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai faiss-cpu

In [33]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./assets-resources/papers/tree-of-thoughts.pdf")
documents = loader.load()

In [34]:
for doc in documents:
    print(doc.page_content[:400])

Tree of Thoughts: Deliberate Problem Solving
with Large Language Models
Shunyu Yao
Princeton UniversityDian Yu
Google DeepMindJeffrey Zhao
Google DeepMindIzhak Shafran
Google DeepMind
Thomas L. Grifﬁths
Princeton UniversityYuan Cao
Google DeepMindKarthik Narasimhan
Princeton University
Abstract
Language models are increasingly being deployed for general problem solving
across a wide range of tasks
GĮŔũƜ
jũƜŔũƜGĮŔũƜ
jũƜŔũƜʱÊʲˤGjʱæʲˤĵÉGĮŔũƜ
ˤjũƜŔũƜʱçʲˤĵÉˁʟʟʟʟaÊĠĵŗƓŤƆˤſĵŤòGĮŔũƜ
ˤjũƜŔũƜʱíʲˤÉĵÉˤʱĵũŗŝʲʟʟʟʟʟʟˤˤʝˤƛĎĵũĈĎƜ)L[FRORUE\<XTLDQ0DUNGLIIHUHQFHE\FRORU
GĮŔũƜ
jũƜŔũƜGĮŔũƜ
jũƜŔũƜGĮŔũƜ
ˤjũƜŔũƜʱçʲˤòĦƙˤĵĮŝƓŝŤòĮçƆˤƀƓƜĎˤĵÉˤʱĵÉˁʲaÊĠĵŗƓŤƆˤſĵŤòGĮŔũƜ
ˤjũƜŔũƜʱíʲˤÉŗòòˤĵƙˤĎĵũĈĎŤŝˤʱÉĵÉʲʟʟʟʟʟʟʟʟʟʟˤˤƛĎĵũĈĎƜ
ʱçʲˤĎÊđĮˤĵƙˤĎĵũĈĎƜˤŗĵĭŔƜđĮĈˤʱĵÉʲʱÊʲˤGĮŔũƜˁjũƜŔũƜˤŗĵĭŔƜđĮĈˤʱGjʲFigure 1: Schematic
Chain-of-thought (CoT) prompting [35] was proposed to address cases where the mapping of
inputxto outputyis non-trivial (e.g. when xis a math question and yis the ﬁnal numerical answer).
The key ide

In [35]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [36]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [37]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)

In [38]:
retriever = db.as_retriever()

from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "tree_of_thoughts",
    "Searches and returns excerpts from the tree of thoughts paper.",
)
tools = [tool]

In [39]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [40]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o', temperature=0)

llm.invoke("Are you tired?")

AIMessage(content="As an AI, I don't have feelings or physical sensations, so I don't experience tiredness. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 11, 'total_tokens': 37, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_72bbfa6014', 'finish_reason': 'stop', 'logprobs': None}, id='run-c1c45aed-2d20-489e-9b7c-a94e3dcede82-0', usage_metadata={'input_tokens': 11, 'output_tokens': 26, 'total_tokens': 37})

In [41]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [42]:
result = agent_executor.invoke(
    {
        "input": "what is the tree of thoughts technique?"
    }
)

result["output"]

'The Tree of Thoughts (ToT) technique is a framework designed to enhance the problem-solving capabilities of large language models (LLMs) by allowing them to explore multiple reasoning paths and make deliberate decisions. This approach generalizes the "Chain of Thought" (CoT) method by enabling exploration over coherent units of text, referred to as "thoughts," which serve as intermediate steps toward solving a problem.\n\nKey aspects of the Tree of Thoughts technique include:\n\n1. **Deliberate Decision Making**: ToT allows language models to consider multiple reasoning paths, self-evaluate choices, and decide on the next course of action. It also incorporates lookahead and backtracking to make global choices.\n\n2. **Problem Framing as a Search**: Problems are framed as a search over a tree, where each node represents a partial solution. The branches of the tree correspond to different thought processes or reasoning paths.\n\n3. **Thought Decomposition**: The intermediate process is 

In [43]:
result = agent_executor.invoke(
    {"input": "How did they evaluated the performance of the technique?"}
)

In [44]:
result["output"]

'The performance of the Tree of Thoughts (ToT) technique was evaluated through a series of experiments involving tasks that challenge existing language model inference methods, even with state-of-the-art models like GPT-4. The tasks included:\n\n1. **Game of 24**: This task required mathematical reasoning to reach a target number using a set of given numbers and operations.\n\n2. **Creative Writing**: This task involved generating coherent and creative text based on given prompts.\n\n3. **Mini Crosswords**: This task involved solving 5x5 mini crosswords, which required lexical reasoning and systematic planning.\n\nThe evaluation involved comparing the ToT approach with other methods such as Input-Output (IO) prompting and Chain-of-Thought (CoT) prompting. The performance metrics included success rates in terms of correct letters, words, and games for the Mini Crosswords task, as well as coherency scores for the Creative Writing task.\n\nThe ToT approach demonstrated superior results ac